In [6]:
def foobar(a: int, b: str, c: float = 3.2) -> tuple: pass

In [29]:
import collections
import functools
import inspect

from typing import List
Vector = List[float]

def formatannotation(annotation, base_module=None):
    if getattr(annotation, '__module__', None) == 'typing':
        return repr(annotation).replace('typing.', '')
    if isinstance(annotation, type):
        if annotation.__module__ in ('builtins', base_module):
            return annotation.__qualname__
        return annotation.__module__+'.'+annotation.__qualname__
    return repr(annotation)

def check(func):
    # 获取函数定义的参数
    sig = inspect.signature(func)
    parameters = sig.parameters          # 参数有序字典
    arg_keys = tuple(parameters.keys())   # 参数名称
    for k, v in sig.parameters.items():
        print('{k}: {a!r}'.format(k=k, a=v.annotation))   
        print("\t", formatannotation(v.annotation))
    print("➷", sig.return_annotation)
    
check(foobar)
def foobar2(a: int, b: str, c: Vector) -> tuple: pass
check(foobar2)

a: <class 'int'>
	 int
b: <class 'str'>
	 str
c: <class 'float'>
	 float
➷ <class 'tuple'>
a: <class 'int'>
	 int
b: <class 'str'>
	 str
c: typing.List[float]
	 List[float]
➷ <class 'tuple'>


In [14]:
fun=exec('def foobar_g(a: int, b: str, c: float = 3.2) -> tuple: pass')
print(foobar_g.__annotations__)

{'a': <class 'int'>, 'b': <class 'str'>, 'c': <class 'float'>, 'return': <class 'tuple'>}


In [30]:
from typing import Mapping, Sequence
def Employee(object): pass
def notify_by_email(employees: Sequence[Employee],
                    overrides: Mapping[str, str]) -> None: pass

check(notify_by_email)

employees: typing.Sequence[Employee]
	 Sequence[Employee]
overrides: typing.Mapping[str, str]
	 Mapping[str, str]
➷ None


In [19]:
from typing import List
Vector = List[float]

def scale(scalar: float, vector: Vector) -> Vector:
    return [scalar * num for num in vector]

# typechecks; a list of floats qualifies as a Vector.
new_vector = scale(2.0, [1.0, -4.2, 5.4])

In [22]:
def foo(a, b, *, c, d=10):
    pass

sig = inspect.signature(foo)
for param in sig.parameters.values():
    if (param.kind == param.KEYWORD_ONLY and
                       param.default is param.empty):
        print('Parameter:', param)
        help(param.annotation)

Parameter: c
Help on class _empty in module inspect:

class _empty(builtins.object)
 |  Marker object for Signature.empty and Parameter.empty.
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [31]:
from io import StringIO
from mako.template import Template
from mako.lookup import TemplateLookup
from mako.runtime import Context

# The contents within the ${} tag are evaluated by Python directly, so full expressions are OK:
def render_template(file, ctx):
    mylookup = TemplateLookup(directories=['./'], output_encoding='utf-8', encoding_errors='replace')
    mytemplate = Template(filename='./templates/'+file, module_directory='/tmp/mako_modules', lookup=mylookup)
    mytemplate.render_context(ctx)
    return (buf.getvalue())

In [32]:
buf = StringIO()
ctx = Context(buf, form_name="some_form", slots=["some_slot", "some_other_slot"])
print(render_template('custom_form_action.mako', ctx))

class CustomFormAction(FormAction):
    def name(self):
        return "some_form"

    @staticmethod
    def required_slots(_tracker):
        return ['some_slot', 'some_other_slot']




In [53]:
from typing import Dict, Text, Any, List, Union

from rasa_core_sdk import ActionExecutionRejection
from rasa_core_sdk import Tracker
from rasa_core_sdk.events import SlotSet
from rasa_core_sdk.executor import CollectingDispatcher
from rasa_core_sdk.forms import FormAction, REQUESTED_SLOT

def build_slots(func):    
    # 获取函数定义的参数
    sig = inspect.signature(func)
    parameters = sig.parameters          # 参数有序字典
    arg_keys = tuple(parameters.keys())   # 参数名称
    for k, v in sig.parameters.items():
        print('{k}: {a!r}, {t}'.format(k=k, a=v.annotation, t=formatannotation(v.annotation)))   
    print("➷", sig.return_annotation)
    return func.__name__, list(parameters.keys())

def simple(a: int, b: str, c: Vector) -> tuple: pass
form_name, slots=build_slots(simple)
print(form_name, str(slots))

buf = StringIO()
ctx = Context(buf, form_name=form_name, slots=slots)
clsdef=render_template('custom_form_action.mako', ctx)
print(clsdef)
exec(clsdef)
exec("form=CustomFormAction()")
print(form.required_slots(None))

a: <class 'int'>, int
b: <class 'str'>, str
c: typing.List[float], List[float]
➷ <class 'tuple'>
simple ['a', 'b', 'c']
class CustomFormAction(FormAction):
    def name(self):
        return "simple_form"

    @staticmethod
    def required_slots(_tracker):
        return ['a', 'b', 'c']


['a', 'b', 'c']
